# Optuna para optimizar hiperparámetros

In [6]:
import optuna
from sklearn.datasets import make_regression
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [3]:
X, y, coef = make_regression(n_samples=1000, n_features=15, n_informative=7, noise=1, coef=True)
X_train, X_test = X[:750, :], X[750:, :]
y_train, y_test = y[:750], y[750:]

In [7]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test) 

In [4]:
coef

array([91.57126079,  8.15728285,  0.        , 75.25225827, 62.98360811,
       60.62421488,  0.        ,  0.        , 76.4548471 ,  6.23733614,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ])

## Lasso

In [11]:
def objective(trial):

    hyperparams = {
        "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False]),
        "alpha": trial.suggest_float("alpha", 1e-2, 10)
    }

    model = Lasso(**hyperparams)

    score = cross_val_score(model, X_train_scaled, y_train, scoring="neg_mean_squared_error", cv=5).mean()

    return score

In [12]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

[I 2021-10-02 11:36:59,542] A new study created in memory with name: no-name-dc52915d-7348-4ef2-85b7-eb1889747b7b
[I 2021-10-02 11:36:59,550] Trial 0 finished with value: -325.2603351761829 and parameters: {'fit_intercept': True, 'alpha': 6.54976045790956}. Best is trial 0 with value: -325.2603351761829.
[I 2021-10-02 11:36:59,555] Trial 1 finished with value: -704.5016149763899 and parameters: {'fit_intercept': False, 'alpha': 9.280835726608117}. Best is trial 0 with value: -325.2603351761829.
[I 2021-10-02 11:36:59,561] Trial 2 finished with value: -245.5147373919238 and parameters: {'fit_intercept': True, 'alpha': 5.666076986915445}. Best is trial 2 with value: -245.5147373919238.
[I 2021-10-02 11:36:59,566] Trial 3 finished with value: -312.36075606914955 and parameters: {'fit_intercept': False, 'alpha': 4.6470869251854845}. Best is trial 2 with value: -245.5147373919238.
[I 2021-10-02 11:36:59,573] Trial 4 finished with value: -199.0301041956959 and parameters: {'fit_intercept': F

In [13]:
study.best_params

{'fit_intercept': True, 'alpha': 0.021298809931760466}

In [14]:
model = Lasso(**study.best_params)
model.fit(X_train_scaled, y_train)

Lasso(alpha=0.021298809931760466)

## Ridge

In [15]:
def objective(trial):

    hyperparams = {
        "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False]),
        "alpha": trial.suggest_float("alpha", 1e-2, 10)
    }

    model = Ridge(**hyperparams)

    score = cross_val_score(model, X_train_scaled, y_train, scoring="neg_mean_squared_error", cv=5).mean()

    return score

In [16]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

[I 2021-10-02 11:38:41,967] A new study created in memory with name: no-name-34f16b27-85cf-451f-a813-ef54c4e2e1fa
[I 2021-10-02 11:38:41,984] Trial 0 finished with value: -152.6921222419727 and parameters: {'fit_intercept': False, 'alpha': 7.2552921346436925}. Best is trial 0 with value: -152.6921222419727.
[I 2021-10-02 11:38:41,990] Trial 1 finished with value: -155.10253159715268 and parameters: {'fit_intercept': False, 'alpha': 9.057140526390814}. Best is trial 0 with value: -152.6921222419727.
[I 2021-10-02 11:38:41,996] Trial 2 finished with value: -1.356936787444705 and parameters: {'fit_intercept': True, 'alpha': 1.9844469572648458}. Best is trial 2 with value: -1.356936787444705.
[I 2021-10-02 11:38:42,001] Trial 3 finished with value: -148.29762687275417 and parameters: {'fit_intercept': False, 'alpha': 0.5419439160708558}. Best is trial 2 with value: -1.356936787444705.
[I 2021-10-02 11:38:42,008] Trial 4 finished with value: -6.473029282505928 and parameters: {'fit_intercep

In [17]:
study.best_params

{'fit_intercept': True, 'alpha': 0.014071786651046192}

In [18]:
model = Ridge(**study.best_params)
model.fit(X_train_scaled, y_train)

Ridge(alpha=0.014071786651046192)

## Modelo como hiperparámetro

In [19]:
def objective(trial):

    hyperparams = {
        "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False]),
        "alpha": trial.suggest_float("alpha", 1e-2, 10)
    }

    model = trial.suggest_categorical("model", ["lasso", "ridge"])

    if model == "lasso":

        model = Lasso(**hyperparams)

    else:

        model = Ridge(**hyperparams)

    score = cross_val_score(model, X_train_scaled, y_train, scoring="neg_mean_squared_error", cv=5).mean()

    return score

In [20]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

[I 2021-10-02 11:41:32,511] A new study created in memory with name: no-name-5724f510-bc96-487b-92f6-26c50e88ce4e
[I 2021-10-02 11:41:32,519] Trial 0 finished with value: -2.9705343757794607 and parameters: {'fit_intercept': True, 'alpha': 4.793571739412173, 'model': 'ridge'}. Best is trial 0 with value: -2.9705343757794607.
[I 2021-10-02 11:41:32,525] Trial 1 finished with value: -482.3146666387953 and parameters: {'fit_intercept': True, 'alpha': 8.468569241303525, 'model': 'lasso'}. Best is trial 0 with value: -2.9705343757794607.
[I 2021-10-02 11:41:32,531] Trial 2 finished with value: -148.3663175274616 and parameters: {'fit_intercept': False, 'alpha': 1.0315618993010016, 'model': 'ridge'}. Best is trial 0 with value: -2.9705343757794607.
[I 2021-10-02 11:41:32,537] Trial 3 finished with value: -150.60659383016798 and parameters: {'fit_intercept': False, 'alpha': 5.244526576719333, 'model': 'ridge'}. Best is trial 0 with value: -2.9705343757794607.
[I 2021-10-02 11:41:32,543] Trial

In [21]:
study.best_params

{'fit_intercept': True, 'alpha': 0.016138665783094634, 'model': 'lasso'}

In [22]:
model = Lasso(fit_intercept=study.best_params["fit_intercept"], alpha=study.best_params["alpha"])
model.fit(X_train_scaled, y_train)

Lasso(alpha=0.016138665783094634)

In [24]:
mean_squared_error(y_train, model.predict(X_train_scaled))

0.9611345757521512

In [25]:
mean_squared_error(y_test, model.predict(X_test_scaled))

0.9411907218653706

In [26]:
model.coef_

array([ 9.04702952e+01,  7.93994672e+00,  0.00000000e+00,  7.52590899e+01,
        6.40805894e+01,  6.37021677e+01,  0.00000000e+00,  2.04519621e-02,
        7.31949298e+01,  5.98340315e+00,  4.57304382e-02,  1.24030217e-02,
       -2.84433840e-02,  1.50345449e-02, -3.77029426e-02])

In [27]:
coef

array([91.57126079,  8.15728285,  0.        , 75.25225827, 62.98360811,
       60.62421488,  0.        ,  0.        , 76.4548471 ,  6.23733614,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ])